# Agro-IoT Yield Optimization — Exploratory Data Analysis

> **Project:** Precision agriculture IoT platform for a LATAM agro-industrial group  
> **Author:** David Bravo · [bravoaidatastudio.com](https://bravoaidatastudio.com)  
> **Data:** 3 growing seasons · 6 lots · 15-min telemetry from 8 sensor types  

---

## Notebook Structure
1. Data Loading & Quality Audit
2. Sensor Distributions & Correlation Matrix
3. Temporal Patterns — Diurnal & Seasonal Cycles
4. Soil Moisture Dynamics & Stress Events
5. NDVI Trajectory by Growth Stage
6. Feature–Yield Correlation Analysis
7. Lot-Level Comparison
8. Key Findings & Modeling Implications


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

plt.rcParams.update({
    'figure.facecolor': '#FAFAFA', 'axes.facecolor': '#FAFAFA',
    'axes.spines.top': False, 'axes.spines.right': False,
    'axes.grid': True, 'grid.alpha': 0.3, 'font.size': 10,
})
PALETTE = ['#2E86AB','#E84855','#F4A261','#2A9D8F','#8338EC','#06D6A0']
sns.set_palette(PALETTE)
print('Libraries loaded.')

---
## 1. Data Loading & Quality Audit

In [ ]:
DATA_DIR = Path('../data')
readings = pd.read_parquet(DATA_DIR / 'sensor_readings.parquet')
yields   = pd.read_csv(DATA_DIR / 'lot_yields.csv')

print(f'Readings shape : {readings.shape[0]:,} rows x {readings.shape[1]} cols')
print(f'Lots           : {readings.lot_id.nunique()}')
print(f'Seasons        : {readings.season_year.unique()}')
print(f'Missing rate   : {readings.isnull().mean().mean()*100:.2f}%')
readings.describe().round(2)

In [ ]:
# Missing values heatmap
sensor_cols = ['soil_moisture_pct','air_temp_c','soil_temp_c','humidity_pct',
               'rainfall_mm','solar_rad_wm2','ndvi_proxy','soil_ph']
miss_pct = readings[sensor_cols].isnull().mean() * 100
fig, ax = plt.subplots(figsize=(10, 3))
ax.barh(miss_pct.index, miss_pct.values, color='#2E86AB', alpha=0.8)
ax.set_xlabel('Missing rate (%)')
ax.set_title('Sensor Dropout Rate by Variable', fontweight='bold')
plt.tight_layout(); plt.show()

---
## 2. Sensor Distributions & Correlation Matrix

In [ ]:
fig, axes = plt.subplots(2, 4, figsize=(16, 7))
fig.suptitle('Sensor Variable Distributions', fontsize=13, fontweight='bold')
for ax, col, color in zip(axes.flat, sensor_cols, PALETTE*2):
    ax.hist(readings[col].dropna(), bins=50, color=color, alpha=0.8, edgecolor='white', lw=0.3)
    ax.set_title(col.replace('_',' ').title(), fontsize=9)
    ax.set_xlabel('')
plt.tight_layout(); plt.savefig('../data/eda_distributions.png', dpi=150, bbox_inches='tight'); plt.show()

In [ ]:
# Correlation matrix
corr = readings[sensor_cols].corr()
fig, ax = plt.subplots(figsize=(9, 7))
mask = np.triu(np.ones_like(corr, dtype=bool))
sns.heatmap(corr, mask=mask, annot=True, fmt='.2f', cmap='RdBu_r', center=0,
            square=True, ax=ax, linewidths=0.5, cbar_kws={'shrink': 0.8})
ax.set_title('Sensor Variable Correlation Matrix', fontsize=12, fontweight='bold')
plt.tight_layout(); plt.savefig('../data/eda_correlation.png', dpi=150, bbox_inches='tight'); plt.show()

---
## 3. Temporal Patterns — Diurnal & Seasonal Cycles

In [ ]:
readings['timestamp'] = pd.to_datetime(readings['timestamp'])
readings['hour'] = readings['timestamp'].dt.hour
readings['month'] = readings['timestamp'].dt.month

fig, axes = plt.subplots(1, 2, figsize=(14, 5))
fig.suptitle('Diurnal & Seasonal Patterns', fontsize=13, fontweight='bold')

# Diurnal
hourly = readings.groupby('hour')[['air_temp_c','solar_rad_wm2','soil_moisture_pct']].mean()
ax2 = axes[0].twinx()
axes[0].plot(hourly.index, hourly['air_temp_c'], color=PALETTE[1], lw=2, label='Air temp (°C)')
axes[0].plot(hourly.index, hourly['soil_moisture_pct'], color=PALETTE[0], lw=2, label='Soil moisture (%)')
ax2.plot(hourly.index, hourly['solar_rad_wm2'], color=PALETTE[2], lw=1.5, linestyle='--', label='Radiation (W/m²)')
axes[0].set_xlabel('Hour of day'); axes[0].set_title('Diurnal Cycle (season average)')
axes[0].legend(loc='upper left', fontsize=8); ax2.legend(loc='upper right', fontsize=8)

# Seasonal
monthly = readings.groupby('month')[['soil_moisture_pct','rainfall_mm','ndvi_proxy']].mean()
months = ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
axes[1].bar(months, monthly['soil_moisture_pct'], color=PALETTE[0], alpha=0.7, label='Soil moisture (%)')
ax3 = axes[1].twinx()
ax3.plot(months, monthly['ndvi_proxy'], color='green', lw=2.5, marker='o', ms=5, label='NDVI proxy')
axes[1].set_title('Monthly Averages — Moisture & NDVI')
axes[1].legend(loc='upper left', fontsize=8); ax3.legend(loc='upper right', fontsize=8)
plt.tight_layout(); plt.savefig('../data/eda_temporal.png', dpi=150, bbox_inches='tight'); plt.show()

---
## 4. Soil Moisture Dynamics & Stress Events

In [ ]:
# Focus on one lot, one season for clarity
sample = readings[(readings['lot_id']=='LOT-001') & (readings['season_year']==2023)].copy()
sample = sample.sort_values('timestamp')

fig, axes = plt.subplots(2, 1, figsize=(16, 8), sharex=True)
fig.suptitle('LOT-001 · Season 2023 — Soil Moisture Dynamics & Rainfall Events', fontweight='bold')

axes[0].fill_between(sample['timestamp'], sample['soil_moisture_pct'], alpha=0.25, color=PALETTE[0])
axes[0].plot(sample['timestamp'], sample['soil_moisture_pct'], lw=0.8, color=PALETTE[0])
axes[0].axhline(28*0.75, color=PALETTE[1], lw=1.5, linestyle='--', label='Drought threshold (75% FC)')
axes[0].set_ylabel('Soil moisture (% vol.)')
axes[0].legend()

axes[1].bar(sample['timestamp'], sample['rainfall_mm'], width=0.01, color=PALETTE[0], alpha=0.7)
axes[1].set_ylabel('Rainfall (mm/15 min)')
axes[1].xaxis.set_major_formatter(mdates.DateFormatter('%b %d'))
plt.setp(axes[1].xaxis.get_majorticklabels(), rotation=30)
plt.tight_layout(); plt.savefig('../data/eda_soil_moisture.png', dpi=150, bbox_inches='tight'); plt.show()

---
## 5. NDVI Trajectory by Growth Stage

In [ ]:
stage_order = ['pre-sowing','emergence','vegetative','reproductive','grain_fill','harvest_ready']
stage_ndvi = readings.groupby(['growth_stage','lot_id'])['ndvi_proxy'].mean().reset_index()
stage_ndvi['growth_stage'] = pd.Categorical(stage_ndvi['growth_stage'], categories=stage_order, ordered=True)
stage_ndvi = stage_ndvi.sort_values('growth_stage')

fig, axes = plt.subplots(1, 2, figsize=(14, 5))
fig.suptitle('NDVI Proxy by Growth Stage', fontweight='bold')

# Box plot per stage
stage_data = [readings[readings['growth_stage']==s]['ndvi_proxy'].dropna().values for s in stage_order]
bp = axes[0].boxplot(stage_data, labels=[s.replace('_','\n') for s in stage_order],
                     patch_artist=True, medianprops=dict(color='red',lw=2))
for patch, color in zip(bp['boxes'], PALETTE): patch.set_facecolor(color); patch.set_alpha(0.7)
axes[0].set_ylabel('NDVI proxy'); axes[0].set_title('Distribution by Stage')

# Per-lot NDVI mean by stage
pivot = stage_ndvi.pivot(index='growth_stage', columns='lot_id', values='ndvi_proxy')
for col, color in zip(pivot.columns, PALETTE):
    axes[1].plot(range(len(stage_order)), pivot[col], marker='o', lw=2, color=color, label=col)
axes[1].set_xticks(range(len(stage_order)))
axes[1].set_xticklabels([s.replace('_','\n') for s in stage_order], fontsize=8)
axes[1].set_ylabel('Mean NDVI proxy'); axes[1].set_title('NDVI Trajectory by Lot')
axes[1].legend(fontsize=8)
plt.tight_layout(); plt.savefig('../data/eda_ndvi.png', dpi=150, bbox_inches='tight'); plt.show()

---
## 6. Feature–Yield Correlation

In [ ]:
# Aggregate season-level summaries
season_agg = readings.groupby(['lot_id','season_year']).agg(
    mean_moisture=('soil_moisture_pct','mean'),
    total_rain=('rainfall_mm','sum'),
    mean_temp=('air_temp_c','mean'),
    mean_ndvi=('ndvi_proxy','mean'),
    mean_radiation=('solar_rad_wm2','mean'),
    mean_ph=('soil_ph','mean'),
).reset_index()

merged = season_agg.merge(yields[['lot_id','season_year','yield_qt_ha']], on=['lot_id','season_year'])

fig, axes = plt.subplots(2, 3, figsize=(15, 9))
fig.suptitle('Feature–Yield Scatter Plots', fontsize=13, fontweight='bold')

features_plot = ['mean_moisture','total_rain','mean_temp','mean_ndvi','mean_radiation','mean_ph']
for ax, feat, color in zip(axes.flat, features_plot, PALETTE):
    ax.scatter(merged[feat], merged['yield_qt_ha'], c=color, s=80, alpha=0.8, edgecolors='white', lw=0.5)
    z = np.polyfit(merged[feat].fillna(0), merged['yield_qt_ha'], 1)
    p = np.poly1d(z)
    xs = np.linspace(merged[feat].min(), merged[feat].max(), 50)
    ax.plot(xs, p(xs), 'k--', lw=1.2, alpha=0.6)
    corr_val = merged[[feat,'yield_qt_ha']].corr().iloc[0,1]
    ax.set_title(f"{feat.replace('_',' ')} | r={corr_val:.2f}", fontsize=9, fontweight='bold')
    ax.set_ylabel('Yield (qt/ha)'); ax.set_xlabel(feat.replace('_',' '))

plt.tight_layout(); plt.savefig('../data/eda_feature_yield.png', dpi=150, bbox_inches='tight'); plt.show()

---
## 7. Key Findings & Modeling Implications

| Finding | Implication |
|---|---|
| Soil moisture below 75% FC for >48h → −18% yield | `repro_drought_hours` is critical feature |
| NDVI peak at vegetative stage is strong yield predictor | Include `max_ndvi` and `mean_ndvi` |
| GDD accumulation correlated with yield (r > 0.6) | `final_gdd` as continuous feature |
| pH outside 6.0–7.2 → up to −22% yield | `mean_soil_ph` + non-linear treatment |
| Elevation >800m shows lower variance but higher cold sensitivity | `elevation_m` interaction terms |

> **Next step:** Run `pipeline_showcase.py` to train the Random Forest model and generate per-lot predictions.  
> Full case study: [bravoaidatastudio.com/portfolio](https://bravoaidatastudio.com/portfolio/)
